In [4]:
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number
%matplotlib inline

In [13]:
months = ['april','febrouary','march']
files = ['\listings.csv','\listings0.csv']
# inputPath= r"C:\Users\myrto\Desktop\data\april"
inputPath= r"C:\Users\nikol\Desktop\dataMining_p1\data\data\april"
columnlist = ['id','zipcode','transit','bedrooms','beds','review_scores_rating','number_of_reviews','neighbourhood','name','latitude','longitude','last_review','instant_bookable','host_since','host_response_rate','host_identity_verified','host_has_profile_pic','first_review','description','city','cancellation_policy','bed_type','bathrooms','accommodates','amenities','room_type','property_type','price','availability_365','minimum_nights']

framelist = []

for file in files:
    print(inputPath+file)
    tempfile = pd.read_csv(inputPath+file)
    tempframe = pd.DataFrame(data=tempfile)
    framelist.append(tempframe)

length = len(framelist)-1

for l in range(0,length):
    #train = framelist[0].merge(framelist[l+1],left_on='id',right_on='id')
    train = framelist[0].combine_first(framelist[l+1])

df.to_csv("train.csv");
df = pd.DataFrame(data = train, columns=columnlist)
# df['accomodates']
df

#train = pd.DataFrame(data=train)
#train
#print(train)

C:\Users\nikol\Desktop\dataMining_p1\data\data\april\listings.csv
C:\Users\nikol\Desktop\dataMining_p1\data\data\april\listings0.csv


,id,zipcode,transit,bedrooms,beds,review_scores_rating,number_of_reviews,neighbourhood,name,latitude,...,cancellation_policy,bed_type,bathrooms,accommodates,amenities,room_type,property_type,price,availability_365,minimum_nights
0,10595,11526,Note: 5-day ticket for all the public transpor...,3.0,5.0,97.0,19,Ambelokipi,"96m2, 3BR, 2BA, Metro, WI-FI etc...",37.98888,...,strict_14_with_grace_period,Real Bed,2.0,8,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",Entire home/apt,Apartment,$73.00,222,1
1,10988,11526,Note: 5-day ticket for all the public transpor...,2.0,2.0,92.0,31,Ambelokipi,"75m2, 2-br, metro, wi-fi, cable TV",37.98843,...,strict_14_with_grace_period,Real Bed,1.0,5,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",Entire home/apt,Apartment,$82.00,0,1
2,10990,11526,Note: 5-day ticket for all the public transpor...,1.0,1.0,97.0,29,Ambelokipi,"50m2, Metro, WI-FI, cableTV, more",37.98783,...,strict_14_with_grace_period,Real Bed,1.0,4,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",Entire home/apt,Apartment,$49.00,234,1
3,10993,115 26,Note: 5-day ticket for all the public transpor...,0.0,1.0,96.0,45,Ambelokipi,"Studio, metro, cable tv, wi-fi, etc",37.98861,...,strict_14_with_grace_period,Real Bed,1.0,2,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",Entire home/apt,Apartment,$38.00,226,1
4,10995,11526,Note: 5-day ticket for all the public transpor...,1.0,1.0,95.0,17,Ambelokipi,"47m2, close to metro,cable TV,wi-fi",37.98950,...,strict_14_with_grace_period,Real Bed,1.0,4,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",Entire home/apt,Apartment,$49.00,249,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9656,33819692,105 56,NaN,1.0,1.0,NaN,0,ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ,Lysicrates Monument - Athens 33,37.97412,...,flexible,Real Bed,1.0,2,"{TV,Wifi,""Air conditioning"",Kitchen,Breakfast,...",Entire home/apt,Apartment,$90.00,260,2
9657,33820145,106 81,Omonia metro station is 600 m away from the ap...,2.0,2.0,NaN,0,ΜΟΥΣΕΙΟ-ΕΞΑΡΧΕΙΑ-ΝΕΑΠΟΛΗ,Bright and Stylish Exarchia apartment,37.98527,...,moderate,Real Bed,2.0,6,"{TV,Wifi,""Air conditioning"",Kitchen,Elevator,H...",Entire home/apt,Apartment,$50.00,263,2
9658,33821757,105 54,NaN,2.0,2.0,NaN,0,ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ,Athens21,37.97698,...,flexible,Real Bed,1.0,4,"{TV,Wifi,""Air conditioning"",Breakfast,Elevator...",Private room,Boutique hotel,$200.00,86,1
9659,33821833,105 56,NaN,1.0,1.0,NaN,0,ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ,Tower of Winds,37.97402,...,flexible,Real Bed,1.0,2,"{TV,Wifi,""Air conditioning"",Kitchen,Breakfast,...",Entire home/apt,Apartment,$90.00,260,2
